In [128]:
# Setup: Add project root to path and import utilities
import sys
sys.path.insert(0, '..')

from utils import get_input

# Load input data for day 10
data = get_input(10, example=False)

print(f"Loaded {len(data)} lines of input")
print("Sample data:")
for line in data[:5]:
    print(f"  {line}")

Loaded 167 lines of input
Sample data:
  [###.] (0,3) (0,2,3) (1,3) (0,1,3) {117,15,2,128}
  [...##..] (2,3) (5,6) (0,1,3,4,5) (0,1,2,6) (2,3,4,5,6) (0,1,5) {12,12,13,13,2,31,19}
  [.###.#..#.] (0,1,3,7,8) (0,1,2,3,4,6,7,9) (0,1,3,5,6,8,9) (3,8) (1,6,7,9) (2,8) (1,3,5) (0,6,9) (4,8) (0,1,3,6,7,8,9) (0,2,5,6,8) {51,33,24,38,16,22,58,25,54,44}
  [#.##] (0,1,2,3) (0,2,3) {9,1,9,9}
  [##.###] (0,1,4,5) (0,1,3,5) (0,1,4) (2,3) (0,1,3,4,5) (1,3) {43,59,16,39,38,25}


In [129]:
# Define B matrix and end_state for the linear algebra formulation
import numpy as np
import re
from dataclasses import dataclass

# Parse the data to extract buttons and end_state
@dataclass
class Machine:
    end_state: list[bool]
    buttons: list[tuple[int, ...]]
    joltages: str

machines = []
for line in data:
    part1 = line[line.index('[')+1:line.index(']')]
    end_state = [c == '#' for c in part1]
    part2 = line[line.index(']')+1:line.index('{')].strip()
    buttons = []
    if part2:
        matches = re.findall(r'\(([0-9,\s]+)\)', part2)
        for match in matches:
            button = tuple(int(x.strip()) for x in match.split(','))
            buttons.append(button)
    joltages = line[line.index('{')+1:line.index('}')].strip()
    machines.append(Machine(end_state, buttons, joltages))

# Use first machine for demonstration
machine = machines[0]
n = len(machine.end_state)  # number of positions
m = len(machine.buttons)     # number of buttons

# Build B matrix: B[i,j] = 1 if button j toggles position i
B = np.zeros((n, m), dtype=int)
for j, button in enumerate(machine.buttons):
    for i in button:
        B[i, j] = 1

# Build end_state vector
e = np.array([int(x) for x in machine.end_state], dtype=int)

# Build joltage end_state vector
e_joltage = np.array([int(x) for x in machine.joltages.split(',')], dtype=int)

print(f"Machine 0: {n} positions, {m} buttons")
print(f"\nB matrix (buttons as columns):")
print(B)
print(f"\ne vector (target state): {e}")
print(f"\ne_joltage vector (target joltage state): {e_joltage}")
print(f"\nButtons: {machine.buttons}")

Machine 0: 4 positions, 4 buttons

B matrix (buttons as columns):
[[1 1 0 1]
 [0 0 1 1]
 [0 1 0 0]
 [1 1 1 1]]

e vector (target state): [1 1 1 0]

e_joltage vector (target joltage state): [117  15   2 128]

Buttons: [(0, 3), (0, 2, 3), (1, 3), (0, 1, 3)]


## Linear Algebra Formulation

### The Equation:
**Bv ≡ end_state (mod 2)**

Where:
- **B** = n×m button matrix
  - n = number of positions (state length)
  - m = number of buttons
  - B[i,j] = 1 if button j toggles position i, else 0
  
- **v** = m×1 vector of button press counts
  - v[j] = number of times button j is pressed
  - We seek v ∈ {0,1}ᵐ (since pressing twice = no effect in mod 2)
  
- **end_state** = n×1 target state vector
  - end_state[i] = 1 if position i should be ON, else 0
  
- **start_state** = [0,0,...,0] = starting state (all OFF)

### Why this works:
- Each button press toggles specific positions (XOR operation)
- XOR is addition in GF(2) (binary field with mod 2 arithmetic)
- Starting from s=0, we need: **start_state + Bv ≡ end_state (mod 2)**
- Since start_state=0: **Bv ≡ end_state (mod 2)**

### Solving:
- This is a system of linear equations over GF(2)
- Solution exists iff e is in the column space of B (over GF(2))
- Find v by Gaussian elimination with mod 2 arithmetic
- Optimal v minimizes ||v||₁ (fewest button presses)

In [130]:
# Example: Solve machine 0 using linear algebra
machine = machines[0]

# Build B matrix
n = len(machine.end_state)
m = len(machine.buttons)

B = np.zeros((n, m), dtype=int)
for j, button in enumerate(machine.buttons):
    for i in button:
        B[i, j] = 1

# Build e vector
e = np.array([int(x) for x in machine.end_state])

print(f"Machine 0: {n} positions, {m} buttons")
print(f"\nButton matrix B:")
print(B)
print(f"\nTarget vector e: {e}")
print(f"\To solve: Bv ≡ e (mod 2)")
print(f"Need to find v = [v₀, v₁, v₂, v₃] where vⱼ ∈ {{0,1}}")
print(f"Each vⱼ tells us whether to press button j (1) or not (0)")



Machine 0: 4 positions, 4 buttons

Button matrix B:
[[1 1 0 1]
 [0 0 1 1]
 [0 1 0 0]
 [1 1 1 1]]

Target vector e: [1 1 1 0]
\To solve: Bv ≡ e (mod 2)
Need to find v = [v₀, v₁, v₂, v₃] where vⱼ ∈ {0,1}
Each vⱼ tells us whether to press button j (1) or not (0)


<>:20: SyntaxWarning: invalid escape sequence '\T'
<>:20: SyntaxWarning: invalid escape sequence '\T'
C:\Users\j.slot\AppData\Local\Temp\ipykernel_31228\1785807299.py:20: SyntaxWarning: invalid escape sequence '\T'
  print(f"\To solve: Bv ≡ e (mod 2)")


In [131]:
# Solve Bv ≡ e (mod 2) and find minimal solution

def row_reduce_gf2(augmented_matrix: np.ndarray) -> tuple[np.ndarray, list[int], list[int]]:
    """
    Perform Gaussian elimination over GF(2) to row-reduce the augmented matrix.
    
    Args:
        augmented_matrix: The augmented matrix [B | e] to reduce
        
    Returns:
        tuple: (reduced_matrix, pivot_columns, free_columns)
    """
    n_rows, n_cols = augmented_matrix.shape
    n_button_cols = n_cols - 1  # Exclude the target column
    
    pivot_cols = []
    free_cols = []
    pivot_row = 0
    
    # Forward elimination to row echelon form
    for col in range(n_button_cols):
        # Find pivot in this column
        pivot_found = False
        for row in range(pivot_row, n_rows):
            if augmented_matrix[row, col] == 1:
                # Swap rows to position pivot
                augmented_matrix[[pivot_row, row]] = augmented_matrix[[row, pivot_row]]
                pivot_found = True
                break
        
        if not pivot_found:
            free_cols.append(col)
            continue
        
        pivot_cols.append(col)
        
        # Eliminate all other 1s in this column using XOR (addition in GF(2))
        for row in range(n_rows):
            if row != pivot_row and augmented_matrix[row, col] == 1:
                augmented_matrix[row] = (augmented_matrix[row] + augmented_matrix[pivot_row]) % 2
        
        pivot_row += 1
    
    return augmented_matrix, pivot_cols, free_cols


def check_consistency(augmented_matrix: np.ndarray) -> bool:
    """
    Check if the system is consistent (has a solution).
    
    Args:
        augmented_matrix: Row-reduced augmented matrix [B | e]
        
    Returns:
        bool: True if consistent, False if inconsistent
    """
    n_rows = augmented_matrix.shape[0]
    
    for row in range(n_rows):
        # Check for row of form [0 0 ... 0 | 1] which indicates inconsistency
        if np.all(augmented_matrix[row, :-1] == 0) and augmented_matrix[row, -1] == 1:
            return False
    
    return True


def find_minimal_solution(
    button_matrix: np.ndarray,
    target_vector: np.ndarray,
    augmented_matrix: np.ndarray,
    pivot_cols: list[int],
    free_cols: list[int]
) -> np.ndarray:
    """
    Find the minimal solution by trying all free variable assignments.
    
    Args:
        button_matrix: Original button matrix B
        target_vector: Target state vector e
        augmented_matrix: Row-reduced augmented matrix
        pivot_cols: List of pivot column indices
        free_cols: List of free column indices
        
    Returns:
        np.ndarray: Minimal solution vector v with minimum ||v||₁
    """
    n_buttons = button_matrix.shape[1]
    min_presses = float('inf')
    best_solution = None
    
    # Try all 2^k combinations of free variables
    for free_assignment in range(2 ** len(free_cols)):
        solution = np.zeros(n_buttons, dtype=int)
        
        # Set free variables based on current assignment
        for i, col in enumerate(free_cols):
            solution[col] = (free_assignment >> i) & 1
        
        # Back-substitute for pivot variables
        for i, col in enumerate(pivot_cols):
            solution[col] = augmented_matrix[i, -1]
            for j in range(col + 1, n_buttons):
                solution[col] = (solution[col] - augmented_matrix[i, j] * solution[j]) % 2
        
        # Verify this is a valid solution
        if np.array_equal((button_matrix @ solution) % 2, target_vector):
            num_presses = np.sum(solution)
            if num_presses < min_presses:
                min_presses = num_presses
                best_solution = solution.copy()
    
    return best_solution


def solve_gf2_minimal(
    button_matrix: np.ndarray,
    target_vector: np.ndarray,
    verbose: bool = True
) -> np.ndarray | None:
    """
    Find the minimal solution to Bv ≡ e (mod 2).
    
    Uses Gaussian elimination over GF(2) then tries all assignments to free
    variables to find the solution with minimum ||v||₁ (fewest button presses).
    
    Args:
        button_matrix: n×m matrix where B[i,j] = 1 if button j toggles position i
        target_vector: n×1 target state vector
        verbose: Whether to print detailed output
        
    Returns:
        Minimal solution vector v, or None if no solution exists
    """
    # Create augmented matrix [B | e]
    augmented = np.column_stack([button_matrix.copy(), target_vector.copy()])
    
    if verbose:
        print("Augmented matrix [B | e]:")
        print(augmented)
        print()
    
    # Perform row reduction
    reduced_matrix, pivot_cols, free_cols = row_reduce_gf2(augmented)
    
    if verbose:
        print("Row-reduced augmented matrix:")
        print(reduced_matrix)
        print()
    
    # Check consistency
    if not check_consistency(reduced_matrix):
        if verbose:
            print("❌ System is inconsistent - no solution exists!")
        return None
    
    if verbose:
        print(f"Pivot columns (basic variables): {pivot_cols}")
        print(f"Free columns (free variables): {free_cols}")
        print(f"Number of free variables: {len(free_cols)}")
        print()
    
    # Find minimal solution
    solution = find_minimal_solution(
        button_matrix,
        target_vector,
        reduced_matrix,
        pivot_cols,
        free_cols
    )
    
    if verbose and solution is not None:
        min_presses = np.sum(solution)
        buttons_to_press = [i for i, val in enumerate(solution) if val == 1]
        
        print(f"✅ MINIMAL solution found: v = {solution}")
        print(f"Minimum button presses: {min_presses}")
        print(f"Press button indices: {buttons_to_press}")
        print(f"Press buttons: {[machine.buttons[i] for i in buttons_to_press]}")
        
        # Verify the solution
        result = (button_matrix @ solution) % 2
        print(f"\nVerification: Bv mod 2 = {result}")
        print(f"Target e = {target_vector}")
        print(f"Match: {np.array_equal(result, target_vector)}")
    
    return solution


# Solve for machine 0 with minimal solution
print("Finding MINIMAL solution:")
print("=" * 60)
v = solve_gf2_minimal(B, e)

Finding MINIMAL solution:
Augmented matrix [B | e]:
[[1 1 0 1 1]
 [0 0 1 1 1]
 [0 1 0 0 1]
 [1 1 1 1 0]]

Row-reduced augmented matrix:
[[1 0 0 0 0]
 [0 1 0 0 1]
 [0 0 1 0 1]
 [0 0 0 1 0]]

Pivot columns (basic variables): [0, 1, 2, 3]
Free columns (free variables): []
Number of free variables: 0

✅ MINIMAL solution found: v = [0 1 1 0]
Minimum button presses: 2
Press button indices: [1, 2]
Press buttons: [(0, 2, 3), (1, 3)]

Verification: Bv mod 2 = [1 1 1 0]
Target e = [1 1 1 0]
Match: True


In [132]:
# do for all machines using solve_gf2_minimal
sum_of_fewest_presses = 0
for i, machine in enumerate(machines):
    # Build B matrix
    n = len(machine.end_state)
    m = len(machine.buttons)

    B = np.zeros((n, m), dtype=int)
    for j, button in enumerate(machine.buttons):
        for k in button:
            B[k, j] = 1

    # Build e vector
    e = np.array([int(x) for x in machine.end_state])

    print(f"\nSolving Machine {i}: {n} positions, {m} buttons")
    v = solve_gf2_minimal(B, e)
    if v is not None:
        num_presses = np.sum(v)
        sum_of_fewest_presses += num_presses
        print(f"Machine {i}: Minimum presses = {num_presses}")
    else:
        print(f"Machine {i}: No solution found")


# Final result: sum of fewest presses for all machines
print(f"\nSum of fewest presses for all machines: {sum_of_fewest_presses}")


Solving Machine 0: 4 positions, 4 buttons
Augmented matrix [B | e]:
[[1 1 0 1 1]
 [0 0 1 1 1]
 [0 1 0 0 1]
 [1 1 1 1 0]]

Row-reduced augmented matrix:
[[1 0 0 0 0]
 [0 1 0 0 1]
 [0 0 1 0 1]
 [0 0 0 1 0]]

Pivot columns (basic variables): [0, 1, 2, 3]
Free columns (free variables): []
Number of free variables: 0

✅ MINIMAL solution found: v = [0 1 1 0]
Minimum button presses: 2
Press button indices: [1, 2]
Press buttons: [(0, 2, 3), (1, 3)]

Verification: Bv mod 2 = [1 1 1 0]
Target e = [1 1 1 0]
Match: True
Machine 0: Minimum presses = 2

Solving Machine 1: 7 positions, 6 buttons
Augmented matrix [B | e]:
[[0 0 1 1 0 1 0]
 [0 0 1 1 0 1 0]
 [1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1]
 [0 0 1 0 1 0 1]
 [0 1 1 0 1 1 0]
 [0 1 0 1 1 0 0]]

Row-reduced augmented matrix:
[[1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 1 0 1 0 1]
 [0 0 0 1 1 0 0]
 [0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]

Pivot columns (basic variables): [0, 1, 2, 3, 5]
Free columns (free variables): [4]
Number of free variables: 1



## Part 2

In [148]:
from scipy.optimize import milp, LinearConstraint, Bounds
import numpy as np

def solve_joltage_ilp(
    button_matrix: np.ndarray,
    joltage_vector: np.ndarray,
    verbose: bool = True
) -> np.ndarray | None:
    """
    Find the minimal solution to Bv = e_joltage with non-negative integer v.
    
    This uses Mixed Integer Linear Programming (MILP) to minimize the total number 
    of button presses subject to the constraint that each position is toggled 
    exactly the required number of times.
    
    Args:
        button_matrix: n×m matrix where B[i,j] = 1 if button j toggles position i
        joltage_vector: n×1 vector of exact toggle counts required for each position
        verbose: Whether to print detailed output
        
    Returns:
        Minimal solution vector v, or None if no solution exists
    """
    n_positions, n_buttons = button_matrix.shape
    
    if verbose:
        print(f"Solving MILP: minimize ||v||₁ subject to Bv = e_joltage")
        print(f"B shape: {button_matrix.shape}")
        print(f"e_joltage: {joltage_vector}")
        print()
    
    # Objective: minimize sum of all button presses
    c = np.ones(n_buttons)
    
    # Equality constraints: Bv = e_joltage
    # Convert to float to ensure proper constraint handling
    joltage_float = joltage_vector.astype(float)
    constraints = LinearConstraint(button_matrix, joltage_float, joltage_float)
    
    # Bounds: v[j] >= 0 (non-negative integers)
    bounds = Bounds(0, np.inf)
    
    # All variables must be integers
    integrality = np.ones(n_buttons)
    
    if verbose:
        print("Solving with MILP solver...")
    
    # Solve as integer program
    result = milp(c=c, constraints=constraints, bounds=bounds, integrality=integrality)
    
    if not result.success:
        if verbose:
            print(f"❌ No solution found: {result.message}")
        return None
    
    # Extract integer solution - use round instead of astype to handle floating point precision
    solution = np.round(result.x).astype(int)
    result_joltage = button_matrix @ solution
    
    # Verify solution
    if not np.array_equal(result_joltage, joltage_vector):
        # Try different rounding strategies
        solution_floor = np.floor(result.x).astype(int)
        result_floor = button_matrix @ solution_floor
        
        solution_ceil = np.ceil(result.x).astype(int)
        result_ceil = button_matrix @ solution_ceil
        
        if np.array_equal(result_floor, joltage_vector):
            solution = solution_floor
            result_joltage = result_floor
        elif np.array_equal(result_ceil, joltage_vector):
            solution = solution_ceil
            result_joltage = result_ceil
        else:
            if verbose:
                print(f"⚠️ Solution verification failed")
                print(f"  MILP result.x: {result.x}")
                print(f"  Rounded: Bv = {result_joltage}")
                print(f"  Floor: Bv = {result_floor}")
                print(f"  Ceil: Bv = {result_ceil}")
                print(f"  e_joltage = {joltage_vector}")
                print(f"  Difference (round): {result_joltage - joltage_vector}")
            return None
    
    if verbose:
        total_presses = np.sum(solution)
        buttons_pressed = [(i, solution[i]) for i in range(n_buttons) if solution[i] > 0]
        
        print(f"✅ Solution found!")
        print(f"v = {solution}")
        print(f"Total button presses: {total_presses}")
        print(f"Buttons pressed: {buttons_pressed}")
        print()
        print(f"Verification: Bv = {result_joltage}")
        print(f"Target:       e_joltage = {joltage_vector}")
        print(f"Match: {np.array_equal(result_joltage, joltage_vector)}")
    
    return solution


# Test with machine 0
print("=" * 70)
print("Testing Machine 0")
print("=" * 70)

# Rebuild B and e_joltage for machine 0
machine_0 = machines[0]
n_pos = len(machine_0.end_state)
n_btn = len(machine_0.buttons)

B_test = np.zeros((n_pos, n_btn), dtype=int)
for j, button in enumerate(machine_0.buttons):
    for i in button:
        B_test[i, j] = 1

e_joltage_test = np.array([int(x) for x in machine_0.joltages.split(',')], dtype=int)

v_solution = solve_joltage_ilp(B_test, e_joltage_test, verbose=True)

Testing Machine 0
Solving MILP: minimize ||v||₁ subject to Bv = e_joltage
B shape: (4, 4)
e_joltage: [117  15   2 128]

Solving with MILP solver...
✅ Solution found!
v = [111   2  11   4]
Total button presses: 128
Buttons pressed: [(0, np.int64(111)), (1, np.int64(2)), (2, np.int64(11)), (3, np.int64(4))]

Verification: Bv = [117  15   2 128]
Target:       e_joltage = [117  15   2 128]
Match: True


In [150]:
# Solve Part 2 for all machines
print("\n" + "=" * 70)
print("Part 2: Solving all machines with joltage constraints")
print("=" * 70 + "\n")

total_presses_part2 = 0

for idx, machine in enumerate(machines):
    # Build button matrix
    n_positions = len(machine.end_state)
    n_buttons = len(machine.buttons)
    B_machine = np.zeros((n_positions, n_buttons), dtype=int)
    
    for j, button in enumerate(machine.buttons):
        for pos in button:
            B_machine[pos, j] = 1
    
    # Parse joltages
    e_joltage_machine = np.array([int(x) for x in machine.joltages.split(',')], dtype=int)
    
    print(f"Machine {idx}:")
    print(f"  Positions: {n_positions}, Buttons: {n_buttons}")
    print(f"  Joltage target: {e_joltage_machine}")
    
    # Solve
    v_solution = solve_joltage_ilp(B_machine, e_joltage_machine, verbose=False)
    
    if v_solution is not None:
        num_presses = np.sum(v_solution)
        total_presses_part2 += num_presses
        print(f"  ✅ Solution: {num_presses} button presses")
    else:
        print(f"  ❌ No solution found")
    
    print()

print("=" * 70)
print(f"Part 2 Answer: Total button presses = {total_presses_part2}")
print("=" * 70)


Part 2: Solving all machines with joltage constraints

Machine 0:
  Positions: 4, Buttons: 4
  Joltage target: [117  15   2 128]
  ✅ Solution: 128 button presses

Machine 1:
  Positions: 7, Buttons: 6
  Joltage target: [12 12 13 13  2 31 19]
  ✅ Solution: 42 button presses

Machine 2:
  Positions: 10, Buttons: 11
  Joltage target: [51 33 24 38 16 22 58 25 54 44]
  ✅ Solution: 80 button presses

Machine 3:
  Positions: 4, Buttons: 2
  Joltage target: [9 1 9 9]
  ✅ Solution: 9 button presses

Machine 4:
  Positions: 6, Buttons: 6
  Joltage target: [43 59 16 39 38 25]
  ✅ Solution: 75 button presses

Machine 5:
  Positions: 10, Buttons: 8
  Joltage target: [24 40 67 26 42 34 62 64 81 64]
  ✅ Solution: 101 button presses

Machine 6:
  Positions: 9, Buttons: 9
  Joltage target: [44 44 44 36 12 42 46 36 27]
  ✅ Solution: 80 button presses

Machine 7:
  Positions: 5, Buttons: 6
  Joltage target: [32 32  0 21 27]
  ✅ Solution: 32 button presses

Machine 8:
  Positions: 7, Buttons: 8
  Joltage

In [151]:
# Display final Part 2 answer
print("\n" + "=" * 70)
print(f"FINAL PART 2 ANSWER: {total_presses_part2}")
print("=" * 70)


FINAL PART 2 ANSWER: 16386
